## Copy and paste for lemur identification

In [46]:
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageDraw
from random import sample, randrange
import pandas as pd
import os
import re
import json
import csv

import sys
if '/usr/users/vogg/.conda/envs/fairmot/lib/python3.8/site-packages' not in sys.path:
    sys.path.insert(0, '/usr/users/vogg/.conda/envs/fairmot/lib/python3.8/site-packages')
    sys.path.insert(0, '/usr/users/vogg/FairMOT/src')
    sys.path.insert(0, '/usr/users/vogg/FairMOT/src/lib')


from pathlib import Path
import matplotlib.pyplot as plt
import shutil

import _init_paths
from models.model import create_model, load_model
from models.decode import mot_decode, map2orig
from tracking_utils.utils import *
from tracking_utils.log import logger
from models import *
from utils.post_process import ctdet_post_process
from utils.image import get_affine_transform
from models.utils import _tranpose_and_gather_feat
import datasets.dataset as datasets

# MC Mot imports
from collections import defaultdict

In [50]:
img_path = Path("/usr/users/vogg/Labelling/Lemurs/Individual_imgs/images/")
background_root = Path("/usr/users/vogg/background_imgs/")

img_name_list = os.listdir(img_path)
bg_list = os.listdir(background_root)
regex = re.compile('.*jpg')
img_name_list = sorted([i for i in img_name_list if regex.match(i)])
bg_list = sorted([i for i in bg_list if regex.match(i)])
bg_list = sorted([i for i in bg_list if not i.startswith(".")])
print(len(img_name_list))

226


In [51]:
model = create_model('hrnet_32', heads =  {'hm': 2, 'wh': 2, 'id': 128, 'reg': 2}, 
                     head_conv = 256, num_classes = 2, num_poses = None, 
                     clsID4Pose = None, cat_spec_wh = False, conf_thres = 0.1)

model = load_model(model, '../models/hrnet/hrnet32_lemur_sep22.pth')
model = model.to(torch.device('cuda'))
model.eval()
print('ok')

=> loading pretrained model /usr/users/vogg/FairMOT/models/hrnetv2_w32_imagenet_pretrained.pth
ok


In [133]:
img_name_list[0]

'GX010011_small_4396.jpg'

In [155]:
for rep in range(10):
    print(rep)
    for i in range(len(img_name_list)):

        img0 = cv2.imread(str(img_path / img_name_list[i]))  # BGR
        img_box, _, _, _ = datasets.letterbox(img0, height=608, width=1088)
        img1 = Image.fromarray(cv2.cvtColor(img0, cv2.COLOR_BGR2RGB))
        # Normalize RGB
        img = img_box[:, :, ::-1].transpose(2, 0, 1)

        img = np.ascontiguousarray(img, dtype=np.float32)
        img /= 255.0

        im_blob = torch.from_numpy(img).cuda().unsqueeze(0)

        width = img0.shape[1]
        height = img0.shape[0]
        inp_height = im_blob.shape[2]
        inp_width = im_blob.shape[3]

        with torch.no_grad():
            output = model(im_blob)[-1]
            hm = output['hm'].sigmoid_()
            #hm = hm * self.prediction_hm
            wh = output['wh']


            reg = output['reg']


        dets, inds, cls_inds_mask = mot_decode(heatmap=hm,
                                                   wh=wh,
                                                   reg=reg,
                                                   num_classes=2,
                                                   cat_spec_wh=False,
                                                   K=5)
        h_out = inp_height // 4
        w_out = inp_width // 4
        dets = map2orig(dets, h_out, w_out, height, width, 2)

        coords = dets[0][0]

        center = tuple(map(int, ((coords[0] + coords[2]) / 2, (coords[1] + coords[3]) / 2)))
        low_left = tuple(map(int, (coords[0], coords[1])))
        up_right = tuple(map(int, (coords[2], coords[3])))
        w = coords[2] - coords[0]
        h = coords[3] - coords[1]

        mask = Image.new('L', (width, height), 0)
        path = [low_left, tuple([coords[0] + w, coords[1]]), 
                tuple([coords[0] + w, coords[1] + h]), tuple([coords[0], coords[1] + h])]
        ImageDraw.Draw(mask).polygon(path, outline=1, fill=255)

        img2 = Image.open(background_root / bg_list[(i + rep) % len(bg_list)])

        x, y = img2.size

        x0, y0 = img1.size

        scale_factor = randrange(50, 95)/100
        width = int(min(x * scale_factor, x0))
        height = int(min(y * scale_factor, y0))
        x1 = randrange(0, x - width)
        y1 = randrange(0, y - height)

        img3 = img2.crop((x1, y1, x1 + width, y1 + height)).resize(img1.size)
        im = Image.composite(img1, img3, mask)
        out_name = str(rep) + "_" + img_name_list[i].split(".")[0] + ".jpg"
        im.save(img_path / out_name)

0
100
200
0
100
200
0
100
200
0
100
200
0
100
200
0
100
200
0
100
200
0
100
200
0
100
200
0
100
200


In [164]:
label_path = Path("/usr/users/vogg/Labelling/Lemurs/Individual_imgs/labels_with_ids/")
file_list = os.listdir(label_path)
file_list = sorted([i for i in file_list if not i.startswith(".")])

In [165]:
import shutil

for rep in range(10):
    print(rep)
    for file in file_list:
        out_file = str(rep) + "_" + file
        shutil.copyfile(str(label_path / file), str(label_path / out_file))

0
1
2
3
4
5
6
7
8
9
